In [1]:
import math
from abc import ABC, abstractmethod

In [2]:
# Base class for term structure models
class TermStructureClass(ABC):
    @abstractmethod
    def d(self, t):
        """Abstract method to be implemented by subclasses."""
        pass

    def r(self, t):
        """Calculate the yield at time t."""
        return -math.log(self.d(t)) / t

    def f(self, t1, t2):
        """Calculate the forward rate between t1 and t2."""
        d1 = self.d(t1)
        d2 = self.d(t2)
        return -math.log(d2 / d1) / (t2 - t1)

In [3]:
# Vasicek term structure model class
class VasicekTermStructure(TermStructureClass):
    def __init__(self, r0, a, b, sigma):
        """
        Initialize the Vasicek model parameters.
        :param r0: short-term rate
        :param a: speed of mean reversion
        :param b: long-run mean rate
        :param sigma: volatility
        """
        self.r0 = r0  # Renamed to r0 to avoid conflict with method r
        self.a = a
        self.b = b
        self.sigma = sigma

    def d(self, t):
        """
        Calculate the discount factor using the Vasicek model.
        :param t: time to maturity
        :return: discount factor
        """
        sigma_sqr = self.sigma ** 2
        a_sqr = self.a ** 2

        if self.a == 0.0:
            B = t
            A = math.exp(sigma_sqr * t ** 3 / 6.0)
        else:
            B = (1.0 - math.exp(-self.a * t)) / self.a
            A = math.exp(
                ((B - t) * (a_sqr * self.b - 0.5 * sigma_sqr)) / a_sqr
                - (sigma_sqr * B ** 2) / (4 * self.a)
            )

        return A * math.exp(-B * self.r0)

In [4]:
def vasicek_discount_factor(time, r, a, b, sigma):
    """
    Calculate the Vasicek model discount factor.
    
    Parameters:
    - time: Time to maturity (float)
    - r: Short-term interest rate (float)
    - a: Speed of mean reversion (float)
    - b: Long-run mean interest rate (float)
    - sigma: Volatility (float)
    
    Returns:
    - Discount factor at time t (float)
    """
    sigma_sqr = sigma ** 2  # sigma squared
    aa = a ** 2  # a squared
    
    if a == 0.0:
        B = time
        A = math.exp(sigma_sqr * time ** 3 / 6.0)
    else:
        B = (1.0 - math.exp(-a * time)) / a
        A = math.exp(((B - time) * (aa * b - 0.5 * sigma_sqr)) / aa - (sigma_sqr * B ** 2) / (4 * a))
    
    # Calculate the discount factor
    discount_factor = A * math.exp(-B * r)
    return discount_factor

In [5]:
# Test the implementation with the provided data
if __name__ == "__main__":
    # Vasicek model parameters
    r0 = 0.05  # Short-term rate
    a = -0.1   # Speed of mean reversion
    b = 0.1   # Long-run mean rate
    sigma = 0.1  # Volatility

    # Instantiate the Vasicek term structure model
    vasicek_model = VasicekTermStructure(r0, a, b, sigma)
    direct_discount_factor = vasicek_discount_factor(1, r0, a, b, sigma)

    # Test values
    t = 1.0  # Time to maturity
    t1, t2 = 1.0, 2.0  # Time range for forward rate

    # Perform calculations
    discount_factor = vasicek_model.d(t)
    yield_at_t = vasicek_model.r(t)
    forward_rate = vasicek_model.f(t1, t2)

    # Output results
    print("Example term structure calculations using the Vasicek model:")
    print(f"Direct discount factor at t={t}: {direct_discount_factor:.6f}")
    print(f"Using Class")
    print(f"Discount factor (t=1): {discount_factor:.6f}")
    print(f"Yield (t=1): {yield_at_t:.6f}")
    print(f"Forward rate (t1=1, t2=2): {forward_rate:.6f}")

Example term structure calculations using the Vasicek model:
Direct discount factor at t=1.0: 0.955408
Using Class
Discount factor (t=1): 0.955408
Yield (t=1): 0.045617
Forward rate (t1=1, t2=2): 0.028148
